# Assignment 5

**Assignment 5: Neuroevolution: Neural Architecture**

**Goal**: Get familiar with how neuroevolution is used for neural network architecture optimization by optimizing a neural network using neuroevolution.

## 1 Understanding the problem



*Problem description goes here*

In [11]:
import numpy as np
import torch
import torch.nn as nn
import scipy.stats as ss
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_digits

In [2]:
class Dataset_:
    class Digits(Dataset):
        def __init__(self, mode, transforms):
            self.digits = load_digits()
            if mode == 'train':
                self.data = self.digits.data[:1000].astype(np.float32)
                self.targets = self.digits.target[:1000]
            elif mode == 'val':
                self.data = self.digits.data[1000:1350].astype(np.float32)
                self.targets = self.digits.target[1000:1350]
            else:
                self.data = self.digits.data[1350:].astype(np.float32)
                self.targets = self.digits.target[1350:]
                
            self.transforms = transforms

        def __len__(self):
            return len(self.data)

        def __getitem__(self, idx):
            sample_x = self.data[idx].reshape(-1, 8, 8)
            sample_y = self.targets[idx]
            if self.transforms:
                sample_x = self.transforms(sample_x)
            return (sample_x, sample_y)
    
    def select_mode(self, mode, transforms=None):
        return self.Digits(mode, transforms)

In [10]:
class PhenotypeExpresser:
    class ClassifierNeuralNet(nn.Module):
        def __init__(self, classnet):
            super(PhenotypeExpresser.ClassifierNeuralNet, self).__init__()
            self.classnet = classnet
            self.nll = nn.NLLLoss(reduction='none')

        def classify(self, x):
            # Return the indices with the highest probability which
            # are equivalent to the class labels because the labels are digits
            return torch.argmax(self.classnet(x), axis=1)

        def forward(self, x, y, reduction='avg'):
            # Calculate the negative log likelihood loss based on the target values y
            # and how the classifier network classifies data x
            loss = self.nll(self.classnet(x), y.type(torch.LongTensor))
            if reduction == 'sum':
                return loss.sum()
            return loss.mean()
    
    def __init__(self):
        self._activations = [nn.ReLU(), nn.Sigmoid(), nn.Tanh(), nn.Softplus(), nn.ELU()]
        self._filters = [8, 16, 32]
    
    def __get_conv_layer(self, filters, structure):
        if structure: kernel, padding = 3, 1
        else: kernel, padding = 5, 2
        stride = 1
        return nn.Conv2d(1, self._filters[filters], kernel_size=kernel, padding=padding, stride=stride)
    
    @staticmethod
    def __get_pool_layer(kernel, avg_pool):
        kernel += 1
        return nn.AvgPool2d(kernel) if avg_pool else nn.MaxPool2d(kernel)
    
    def __get_input_size(self, filters, structure, kernel_pool):
        if structure: kernel, padding = 3, 1
        else: kernel, padding = 5, 2
        kernel_pool += 1
        return int(self._filters[filters]*((8-kernel+2*padding+1)/kernel_pool)**2)
    
    def get_phenotypes(self, genotypes):
        phenotypes = []
        for genotype in genotypes:
            classnet = nn.Sequential(
                self.__get_conv_layer(*genotype[:2]),
                self._activations[genotype[2]],
                self.__get_pool_layer(*genotype[3:5]),
                nn.Flatten(),
                nn.Linear(self.__get_input_size(*genotype[np.asarray([0,1,3])]), genotype[5]*10),
                self._activations[genotype[2]],
                nn.Linear(genotype[5]*10, 10),
                nn.LogSoftmax(dim=1))
            
            phenotypes.append(self.ClassifierNeuralNet(classnet))
        return np.asarray(phenotypes)
            

In [189]:
class Evaluator:
    def __init__(self, epochs=30, lr=1e-3, wd=1e-5):
        self.PE = PhenotypeExpresser()
        dataset = Dataset_()
        [train, val, test] = [dataset.select_mode(mode) for mode in ['train', 'val', 'test']]
        self.training_loader = DataLoader(train, batch_size=64, shuffle=True)
        self.val_loader = DataLoader(val, batch_size=64, shuffle=False)
        self.test_loader = DataLoader(test, batch_size=64, shuffle=False)
        self.epochs = epochs
        self.lr = lr
        self.wd = wd
    
    def test(self, model, test_loader):
        model.eval()
        loss_test = 0.
        loss_error = 0.
        N = 0.
        
        # start evaluation
        for indx_batch, (test_batch, test_targets) in enumerate(test_loader):
            loss_test_batch = model.forward(test_batch, test_targets, reduction='sum')
            loss_test = loss_test + loss_test_batch.item()
            # classification error
            y_pred = model.classify(test_batch)
            e = 1.*(y_pred == test_targets)
            loss_error = loss_error + (1. - e).sum().item()
            # the number of examples
            N = N + test_batch.shape[0]
            
        loss_test = loss_test / N
        loss_error = loss_error / N

        return loss_test, loss_error
    
    def train(self, model, optimizer):
        nll_val, error_val, beast_nll = [], [], 1000.

        # Main training loop
        for e in range(self.epochs):
            model.train()
            for indx_batch, (batch, targets) in enumerate(self.training_loader):
                loss = model.forward(batch, targets)
                optimizer.zero_grad()
                loss.backward(retain_graph=True)
                optimizer.step()

            # Validation: Evaluate the model on the validation data
            loss_e, error_e = self.test(model, self.val_loader)
            nll_val.append(loss_e)
            error_val.append(error_e)
        
        nll_val, error_val = np.asarray(nll_val), np.asarray(error_val)

        return model
    
    def evaluate(self, x):
        f = []
        x = self.PE.get_phenotypes(x)
        for model in x:
            optimizer = torch.optim.Adam([p for p in model.parameters() if p.requires_grad],
                                         lr=self.lr, weight_decay=self.wd) 
            model = self.train(model, optimizer)
            _, error = self.test(model, self.test_loader)
            f.append(error)
        return np.asarray(f)



In [196]:
class EA:
    def __init__(self, evaluator, pop_size, bounds_min, bounds_max, std=1, parents=None):
        self.evaluator = evaluator
        self.pop_size = pop_size
        self.std = std
        self.parents = parents if parents is not None else pop_size
        self.bounds_min = bounds_min
        self.bounds_max = bounds_max
    
    # Select and return a random number of parents equal to parents hyperparameter
    def parent_selection(self, x_old, f_old):
        ind = np.random.choice(self.pop_size, self.parents, replace=False)
        return x_old, f_old#x_old[ind], f_old[ind]
    
    # Takes features from best parents and inserts them into the worst ones
    def recombination(self, x_parents, f_parents):
        return x_parents
    
    # Add some noise from a multivariate normal distribution to every child
    def mutation(self, x_children):
        binary_mutation = np.random.binomial(1, 0.2, (x_children.shape[0], 3))
        integer_mutation = np.random.binomial(1, 0.2, (x_children.shape[0], 3)) \
                           * np.round(np.random.normal(0, self.std, size=(x_children.shape[0], 3)))
        
        x_children[:,np.array([1, 3, 4])] = (x_children[:,np.array([1, 3, 4])] + binary_mutation)%2 # Flipping bits
        x_children[:,np.array([0, 2, 5])] = x_children[:,np.array([0, 2, 5])] + integer_mutation # Creep mutation

        return np.clip(x_children, self.bounds_min, self.bounds_max)
    
    # Select 80% of survivors based on the round robin tournament selection method
    # The other 20% are selected randomly to help escape local optima
    def survivor_selection(self, x_old, x_children, f_old, f_children):
        x = np.concatenate([x_old, x_children])
        f = np.concatenate([f_old, f_children])
        #scores = self.__winning_scores(f)
        
        ind = np.argsort(f)
        x = x[ind]
        f = f[ind]
        
        return x[:self.pop_size], f[self.pop_size]
        #threshold = int(np.ceil(self.pop_size*0.8))
        #ind = np.random.choice(range(threshold, x.shape[0]), int(0.2*self.pop_size), replace=False)
        #return np.concatenate([x[:threshold], x[ind]]), np.concatenate([f[:threshold], f[ind]])
    
    # For every individual, finds the number of wins against random opponents
    def __winning_scores(self, f):
        scores = []
        for f_i in f:
            ind = np.random.choice(x.shape[0], int(np.ceil(self.pop_size/10)), replace=False)
            scores.append(sum([f_i >= i for i in f[ind]]))
        
        return np.asarray(scores)

    def evaluate(self, x):
        return self.evaluator.evaluate(x)
  
    def step(self, x_old, f_old):
        x_parents, f_parents = self.parent_selection(x_old, f_old)
        x_children = self.recombination(x_parents, f_parents)
        x_children = self.mutation(x_children)
        f_children = self.evaluate(x_children)
        x, f = self.survivor_selection(x_old, x_children, f_old, f_children)
                       
        return x, f

In [197]:
num_generations = 10
pop_size = 20
bounds_min = np.asarray([0, 0, 0, 0, 0, 1])
bounds_max = np.asarray([2, 1, 4, 1, 1, 10])


e = Evaluator()
ea = EA(e, pop_size, bounds_min, bounds_max)


x = np.random.randint(low=bounds_min, high=bounds_max+1, size=(pop_size, 6))
f = e.evaluate(x)

populations = []
populations.append(x)
f_best = [f.min()]

# Run the EA.
for i in range(num_generations):
    if i % int(num_generations * 0.1) == 0:
        print('Generation: {}, best fitness: {:.2f}'.format(i, f.min()))
    x, f = ea.step(x, f)
    populations.append(x)
    if f.min() < f_best[-1]:
        f_best.append(f.min())
    else:
        f_best.append(f_best[-1])
print('FINISHED!')

Generation: 0, best fitness: 0.06
Generation: 1, best fitness: 0.08


ValueError: zero-dimensional arrays cannot be concatenated